In [4]:
import pandas as pd
import os
from pathlib import Path
import cgi
import requests
import json
from simple_salesforce import Salesforce

PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'Keys for Big Query Storage Admin - PEII.json'

'/Users/dhyanna/socos/impact-index/keys/Keys for Big Query Storage Admin - PEII.json'

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [10]:
from google.cloud import bigquery

In [29]:
project = 'peii-242723'
dataset_id = 'entrepreneur_data'

# TODO(developer): Construct a BigQuery client object.
client = bigquery.Client()
dataset_ref = client.dataset(dataset_id, project=project)

#TODO(developer): Set table_id to the ID of the table to create
table_id = "peii-242723.entrepreneur_data.startout"


Created table peii-242723.salesforce_data.startout


In [32]:
sf = Salesforce(username='info@startout.org', password='6MfQ3fYZcx', security_token='GdC0WA2iKWRLD7fP53Sy8dN3')

In [33]:
query = 'SELECT Id, \
         Name, \
         Gender__c, \
         LGBTQ_Identification__c, \
         Racial_Ethnic_Identification__c, \
         Member_Company__c, \
         Name_of_Member_Company__c, \
         MailingCity \
         FROM Contact'

In [34]:
bulk_call = sf.bulk.Contact.query(query)
sf_df = pd.DataFrame(bulk_call)

In [56]:
df = sf_df[sf_df.columns[0:8]]

In [60]:
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("Id", "STRING"),
    bigquery.SchemaField("Name", "STRING"),
    bigquery.SchemaField("MailingCity", "STRING"),
    bigquery.SchemaField("Gender__c", "STRING"),
    bigquery.SchemaField("LGBTQ_Identification__c", "STRING"),
    bigquery.SchemaField("Racial_Ethnic_Identification__c", "STRING"),
    bigquery.SchemaField("Name_of_Member_Company__c", "STRING"),
    bigquery.SchemaField("Member_Company__c", "STRING"),
])

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)

# Wait for the load job to complete.
job.result()